In [1]:
pip install sodapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 597.7 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1

[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sodapy import Socrata
import json

pd.set_option('display.max_columns', None) # show all the columns in the dataframe

# Using the NYC Open Data API to access data

In this file we will:

- create API tokens (you should make your own)
- access the [Community Board Requests Data](https://data.cityofnewyork.us/City-Government/Register-of-Community-Board-Budget-Requests/vn4m-mk4t) data through the API

API documentation [here](https://dev.socrata.com/foundry/data.cityofnewyork.us/mwzb-yiwb)

# Create API Keys

Step 1: Create an [Open Data Account](https://opendata.cityofnewyork.us/)

Step 2: Create API keys through the [developer portal](https://data.cityofnewyork.us/profile/edit/developer_settings)

Step 3: Store your keys in a local file (jsons are usually easy)

Step 4: Read in keys

## To create your credentials file:

your json file should look like this:

{"api_key": "",
 "api_key_secret": "",
 "app_token": "",
 "app_token_secret": ""}

open a text editor (sublime or text edit) and copy the above, then save the file with a .json extention 

don't share this!!! (don't store it on git)

In [15]:
# Read in credentials - stored LOCALLY on your own computer
with open('opendatacredentials.json') as creds:
    credentials = json.load(creds)

api_key = credentials['api_key']
api_key_secret = credentials['api_key_secret']
app_token = credentials['app_token']
app_token_secret = credentials['app_token_secret']

# Access Data

We began to look at data surrounding truck-involved accidents in Red Hook. This data was available through the NYC Open Data [portal vehicle collisions](https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95) data collected by NYPD. The data is queried from 2012 through April 2022. The data is shared using coordinates and cross streets. This data is likely incomplete, since it only tracks police-reported incidents.

In [16]:
dataset_identifier = "h9gi-nx95"
client = Socrata("data.cityofnewyork.us", app_token)

In [17]:
metadata = client.get_metadata(dataset_identifier)

In [18]:
results = client.get(dataset_identifier)

## Put data into a data frame so we can analyze it

In [19]:
df = pd.json_normalize(results)


In [20]:
df.sample(5)

,crash_date,crash_time,on_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,collision_id,vehicle_type_code1,vehicle_type_code2,borough,zip_code,latitude,longitude,cross_street_name,location.latitude,location.longitude,location.human_address,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
85,2021-12-07T00:00:00.000,7:50,CARROLL STREET,NaN,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,4485772,Sedan,Sedan,NaN,NaN,40.66718,-73.95076,NaN,40.66718,-73.95076,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",NaN,NaN,NaN,NaN,NaN,NaN
186,2022-03-26T00:00:00.000,13:28,GRAHAM AVENUE,DEBEVOISE STREET,1,0,0,0,0,0,1,0,Following Too Closely,Unspecified,4514046,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,BROOKLYN,11206,40.701637,-73.942276,NaN,40.701637,-73.942276,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",NaN,NaN,NaN,NaN,NaN,NaN
365,2021-07-08T00:00:00.000,8:50,BROOKLYN QUEENS EXPRESSWAY,NaN,0,0,0,0,0,0,0,0,Unsafe Lane Changing,Unspecified,4456599,Tow Truck / Wrecker,Sedan,NaN,NaN,40.673107,-73.99959,NaN,40.673107,-73.99959,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",NaN,NaN,NaN,NaN,NaN,NaN
78,2021-12-05T00:00:00.000,8:20,NaN,NaN,0,0,0,0,0,0,0,0,Unspecified,NaN,4485408,Sedan,NaN,BROOKLYN,11212,40.658413,-73.9171,393 ROCKAWAY PARKWAY,40.658413,-73.9171,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",NaN,NaN,NaN,NaN,NaN,NaN
367,2021-09-11T00:00:00.000,17:45,BROOKVILLE BOULEVARD,133 AVENUE,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,4455904,Sedan,Tow Truck / Wrecker,QUEENS,11422,40.675255,-73.73207,NaN,40.675255,-73.73207,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df['crash_date'].min()

'2016-04-16T00:00:00.000'

In [22]:
df['crash_date'].max()

'2022-07-12T00:00:00.000'